# Vanderpol model

We consider the following problem :

\begin{equation} 
\left\{ 
\begin{aligned} 
{\mathrm d}_t y_1 & = y_2\\ 
{\mathrm d}_t y_2 & = \varepsilon \, (1 - y_1^2) \, y_2  - y_1 \quad \text{avec } \varepsilon > 0
\end{aligned} 
\right. 
\end{equation}

In [1]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import column
from bokeh.models import PrintfTickFormatter

from ipywidgets import interact, FloatSlider

from mylib.model import vanderpol_model
import mylib.integration as integration

import time

output_notebook(hide_banner=True)

## Quasi exact solution

The quasi-exact solution is obtained by using an implicit Runge-Kutta method of Radau IIA family of order 5 with stepsize control and fine tolerances.

In [2]:
def plot_quasi_exact_sol():

    vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn_radau

    tini = 0. 
    tend = 100.

    yini = (0.5, 0)

    sol_exa = integration.radau5(tini, tend, yini, fcn, 2, rtol=1.e-12, atol=1.e-12)

    fig_sol_exa_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Quasi exact solution : y1")
    plt_y1 = fig_sol_exa_y1.line(sol_exa.t, sol_exa.y[0], line_width=2) 

    fig_sol_exa_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Quasi exact solution : y2")
    plt_y2 = fig_sol_exa_y2.line(sol_exa.t, sol_exa.y[1], line_width=2) 

    show(column(fig_sol_exa_y1, fig_sol_exa_y2), notebook_handle=True)

    def update(eps) :
        vdpm = vanderpol_model(eps)
        fcn = vdpm.fcn_radau  
        sol_exa = integration.radau5(tini, tend, yini, fcn, 2, rtol=1.e-12, atol=1.e-12)
        plt_y1.data_source.data = dict(x=sol_exa.t, y=sol_exa.y[0])
        plt_y2.data_source.data = dict(x=sol_exa.t, y=sol_exa.y[1])
        #plt_pha.data_source.data = dict(x=sol_exa.y[0], y=sol_exa.y[1])
        push_notebook()
        
    interact(update, eps=FloatSlider(min=1.,max=20.,step=1., value=1., continuous_update=False))
    
plot_quasi_exact_sol()

OSError: [WinError 126] 找不到指定的模块。

* y2的速度，当epsilon很大的时候，y2要么不动，要么violent

In [3]:
def plot_radau_cost():

    bm = vanderpol_model(eps=20)
    fcn = bm.fcn
    jac = bm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)

    l_nt = [4001, 5001,8001,10001,20001,40001]
    fe_rk38=[]
    norm_rk38=[]
    ffe_rk38=[]
    
    for i in range(0, len(l_nt)):
        sol_rk38 = integration.rk38(tini, tend, l_nt[i], yini, fcn)
        sol_exa = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
        err = 0 
        sol_local=sol_rk38
        sol=sol_exa
        fe_rk38.append(l_nt[i]-1)
        ffe_rk38.append(4*(l_nt[i]-1))
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)    
        norm_rk38.append(err)


    
    l_tol = [1.e-2, 1.e-3, 1.e-4, 1.e-5, 1.e-6, 1.e-7, 1.e-9, 1.e-10]
    fe_radau=[]
    norm_radau=[]
    ffe_radau=[]
    fe_dopri5=[]
    norm_dopri5=[]
    ffe_dopri5=[]
    fe_embed=[]
    norm_embed=[]
    ffe_embed=[]
    
    time_radau=[]
    time_dopri5=[]
    time_embed=[]
    
    
    for tol in l_tol:
        time0=time.time()
        sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
        time1=time.time()
        time_embed.append(time1-time0)
        
        time0=time.time()
        sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        time1=time.time()
        time_dopri5.append(time1-time0)
        
        time0=time.time()
        sol_radau = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol,jac=jac)
        time1=time.time()
        time_radau.append(time1-time0)

        
        
        sol1 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
        sol2 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t)
        sol3 = solve_ivp(fcn, (tini, tend), yini, rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t)
        ffe_radau.append(sol_radau.nfev)
        ffe_dopri5.append(sol_dopri5.nfev)
        ffe_embed.append(sol_rk_emb.nfev)
        fe_embed.append(sol_rk_emb.t.size-1)
        fe_dopri5.append(sol_dopri5.t.size-1)
        fe_radau.append((sol_radau.t.size-1))
        
        
        
        err = 0 
        sol_local=sol_rk_emb
        sol=sol1
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)    
        norm_embed.append(err)
        
        err = 0 
        sol_local=sol_dopri5
        sol=sol2
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)
        norm_dopri5.append(err)
        
        err = 0 
        sol_local=sol_radau
        sol=sol3
        for i in range(sol_local.t.size-1):
            ldt = (sol_local.t[i+1]-sol_local.t[i])/(tend-tini)
            err += ldt * ((sol.y[0,i+1]-sol_local.y[0,i+1])*(sol.y[0,i+1]-sol_local.y[0,i+1]))
        err = np.sqrt(err)  
        norm_radau.append(err)

    #time steps in function of accuracy 
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of time steps in function of accuracy" )
    fig.x(norm_embed, fe_embed, legend="embedded RK4", color="Indigo")
    fig.line(norm_embed, fe_embed, legend="embedded RK4", color="Indigo")
    fig.x(norm_radau, fe_radau, legend="RADAU", color="Green")
    fig.line(norm_radau, fe_radau, legend="RADAU", color="Green")
    fig.x(norm_dopri5, fe_dopri5, legend="DOPRI5", color="Red")
    fig.line(norm_dopri5, fe_dopri5, legend="DOPRI5", color="Red")
    fig.x(norm_rk38, fe_rk38, legend="RK38", color="Brown")
    fig.line(norm_rk38, fe_rk38, legend="RK38", color="Brown")
    
    show(fig)

    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of time steps in function of tolerance" )
    fig.x(l_tol, fe_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, fe_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, fe_radau, legend="RADAU", color="Green")
    fig.line(l_tol, fe_radau, legend="RADAU", color="Green")
    fig.x(l_tol, fe_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, fe_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Number of function evaluations in function of tolerance" )
    fig.x(l_tol, ffe_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, ffe_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, ffe_radau, legend="RADAU", color="Green")
    fig.line(l_tol, ffe_radau, legend="RADAU", color="Green")
    fig.x(l_tol, ffe_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, ffe_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    fig = figure(x_axis_type="log", y_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Error of y1 as function of tolerance" )
    fig.x(l_tol, norm_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, norm_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, norm_radau, legend="RADAU", color="Green")
    fig.line(l_tol, norm_radau, legend="RADAU", color="Green")
    fig.x(l_tol, norm_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, norm_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
    fig = figure(x_axis_type="log", plot_height=500, plot_width=900, \
                 title= "Real time comsumption as function of tolerance" )
    fig.x(l_tol, time_embed, legend="embedded RK4", color="Indigo")
    fig.line(l_tol, time_embed, legend="embedded RK4", color="Indigo")
    fig.x(l_tol, time_radau, legend="RADAU", color="Green")
    fig.line(l_tol, time_radau, legend="RADAU", color="Green")
    fig.x(l_tol, time_dopri5, legend="DOPRI5", color="Red")
    fig.line(l_tol, time_dopri5, legend="DOPRI5", color="Red")

    show(fig)
    
plot_radau_cost()

## Radau5 integration (python implementation)

In [4]:
def plot_radau5_sol():
    
    vdpm = vanderpol_model(eps=20.)
    fcn = vdpm.fcn
    fcn_radau = vdpm.fcn_radau
    jac = vdpm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)
    
    tol = 1.e-9
    t0 = time.time()
    sol_radau = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=tol, atol=tol, jac=jac)
    t1 = time.time()

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_radau.t, sol_radau.y[0], line_width=2)  
    
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_radau.t, sol_radau.y[1], line_width=2)  
        
    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t, y1_err, line_width=2, legend="y1")
    fig_err.x(sol_radau.t, y2_err, line_width=2, legend="y2", color="crimson")
    fig_err.legend.location = "bottom_right"
    
    show(column(fig_sol_y1, fig_sol_y2, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of accepted steps : {(sol_radau.t.size-1):d}")
    print(f"Number of function evaluations : {sol_radau.nfev:d}")
    print(f"Number of evaluations of the jacobian : {sol_radau.njev:d}")
    print(f"Number of LU decompositions {sol_radau.nlu:d}")

plot_radau5_sol()

Time to integrate : 2.65655 s
Number of accepted steps : 5550
Number of function evaluations : 41624
Number of evaluations of the jacobian : 719
Number of LU decompositions 1926


* radau

## Runge-Kutta "3/8 method"

In [5]:
def plot_rk38_sol():
    
    vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn
    jac = vdpm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)
    
    nt = 10000
    t0 = time.time()
    sol_rk38 = integration.rk38(tini, tend, nt, yini, fcn)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk38.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk38.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk38.y[1])

    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_rk38.t, sol_rk38.y[0], legend="y1", line_width=2)    
    
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_rk38.t, sol_rk38.y[1], line_width=2)  

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk38.t, y1_err, line_width=2, legend="y1")
    fig_err.x(sol_rk38.t, y2_err, line_width=2, legend="y2", color="crimson")
    fig_err.legend.location = "bottom_right"
    
    show(column(fig_sol_y1, fig_sol_y2, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(nt-1):d}")
    print(f"Number of function evaluations : {(4*(nt-1)):d}")

plot_rk38_sol()

Time to integrate : 0.29691 s
Number of time steps : 9999
Number of function evaluations : 39996


* runge kutta avec pas fixe

## Embedded method

In [6]:
def plot_rk_emb_sol():
    
    vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn
    jac = vdpm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)

    tol = 1.e-8
    t0 = time.time()
    sol_rk_emb = integration.rk_embedded(tini, tend, yini, fcn, tol)
    t1 = time.time()

    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_rk_emb.t)
    y1_err = np.abs(sol_exa.y[0] - sol_rk_emb.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_rk_emb.y[1])
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_rk_emb.t, sol_rk_emb.y[0], legend="y1", line_width=2)    
    
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_rk_emb.t, sol_rk_emb.y[1], line_width=2)  

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_rk_emb.t, y1_err, line_width=2, legend="y1")
    fig_err.x(sol_rk_emb.t, y2_err, line_width=2, legend="y2", color="crimson")
    fig_err.legend.location = "bottom_right"
    
    show(column(fig_sol_y1, fig_sol_y2, fig_err))

    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(sol_rk_emb.t.size-1):d}")
    print(f"Number of function evaluations : {sol_rk_emb.nfev:d}")

plot_rk_emb_sol()

Time to integrate : 0.31254 s
Number of time steps : 5358
Number of function evaluations : 21621


## Dopri5 method

In [7]:
def plot_dopri5_sol():
    
    vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn
    jac = vdpm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)
    
    tol = 1.e-8
    t0 = time.time()
    sol_dopri5 = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_dopri5.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_dopri5.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_dopri5.y[1])
        
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_dopri5.t, sol_dopri5.y[0], legend="y1", line_width=2)    
    
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_dopri5.t, sol_dopri5.y[1], line_width=2)  

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_dopri5.t, y1_err, line_width=2, legend="y1")
    fig_err.x(sol_dopri5.t, y2_err, line_width=2, legend="y2", color="crimson")
    fig_err.legend.location = "bottom_right"

    show(column(fig_sol_y1, fig_sol_y2, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")
    print(f"Number of time steps : {(sol_dopri5.t.size-1):d}")
    print(f"Number of function evaluations : {sol_dopri5.nfev:d}")
    
plot_dopri5_sol()

Time to integrate : 0.18754 s
Number of time steps : 1645
Number of function evaluations : 12116


## Radau method (fortran implementation)

In [ ]:
def plot_radau5_fortran_sol():
    
    vdpm = vanderpol_model(eps=1)
    fcn = vdpm.fcn
    fcn_radau = vdpm.fcn_radau
    jac = vdpm.jac
    
    tini = 0. 
    tend = 100.
    
    yini = (0.5, 0)
    
    tol = 1.e-6
    t0 = time.time()
    sol_radau = integration.radau5(tini, tend, yini, fcn_radau, 2, rtol=tol, atol=tol)
    t1 = time.time()
    
    sol_exa = solve_ivp(fcn, (tini, tend), yini, method="Radau", rtol=1.e-12, atol=1.e-12, t_eval=sol_radau.t, jac=jac)
    y1_err = np.abs(sol_exa.y[0] - sol_radau.y[0])
    y2_err = np.abs(sol_exa.y[1] - sol_radau.y[1])

    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    fig_sol.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    fig_sol.legend.location = "top_left"
    
    fig_sol_y1 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y1")
    fig_sol_y1.x(sol_radau.t, sol_radau.y[0], legend="y1", line_width=2)    
    
    fig_sol_y2 = figure(x_range=(tini, tend), width=950, height=300, title="Solution y2")
    fig_sol_y2.x(sol_radau.t, sol_radau.y[1], line_width=2)  

    fig_err = figure(x_range=(tini, tend), y_axis_type="log", plot_height=300, plot_width=950, title="Global error")
    fig_err.yaxis[0].formatter = PrintfTickFormatter(format="%8.1e")
    fig_err.x(sol_radau.t, y1_err, line_width=2, legend="y1")
    fig_err.x(sol_radau.t, y2_err, line_width=2, legend="y2", color="crimson")
    fig_err.legend.location = "bottom_right"
    
    show(column(fig_sol_y1, fig_sol_y2, fig_err))
    
    print(f"Time to integrate : {(t1-t0):.5f} s")    
    print(f"Number of function evalusations : {sol_radau.nfev:d}")
    print(f"Number of jacobian evaluations : {sol_radau.njev:d}")
    print(f"Number of computed steps : {sol_radau.nstep:d}")
    print(f"Number of accepted steps : {sol_radau.naccpt:d}")
    print(f"Number of rejected steps : {sol_radau.nrejct:d}")
    print(f"Number of LU decompositions : {sol_radau.ndec:d}")
    print(f"Number of forward-backward substitutions : {sol_radau.nsol:d}")
    
plot_radau5_fortran_sol()